# Running a simple polarimeter
In this tutorial we will simulate a simple polarimeter. We will implement a polarimeter that can only measure linear polarization states. It will consist of a rotating half-wave plate (the polarization modulator) and a linear polarizer (the analyzer). This polarimeter will be a single-beam system, which means that it can only measure one polarization state at the time. Therefore, it will use temporal polarization modulation to measure the other states as well.

We start by importing the relevant python modules. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 

from hcipy import * 

%matplotlib inline

We will do our measurements with a 4 meter diameter telescope at a wavelength of 500 nanometer. This will set the spatial resolution of the system.  

In [ ]:
# parameters telescope
telescope_diameter = 4 # meter 
wavelength = 500E-9 #m

# the spatial resolution of the telescope
spatial_resolution_telescope = wavelength / telescope_diameter

# parameters star
zero_magnitude_flux = 3.9E10
stellar_magnitude = 0

Setting the pupil- and focal-plane grids. We also define the propagator.

In [ ]:
# setting the grids
pupil_grid = make_pupil_grid(128, telescope_diameter)
focal_grid = make_focal_grid(q=6, num_airy=15, spatial_resolution=spatial_resolution_telescope)

# the propagator between the pupil- and focal-grid. 
propagator = FraunhoferPropagator(pupil_grid, focal_grid) 

# setting the aperture of the telescope 
aperture = circular_aperture(telescope_diameter)(pupil_grid)

Defining the states of the polarization modulator and the linear polarizer. 

In [ ]:
# states of the half-wave plate. 
HWP_state_1 = HalfWavePlate(0)
HWP_state_2 = HalfWavePlate(np.radians(45))
HWP_state_3 = HalfWavePlate(np.radians(22.5))
HWP_state_4 = HalfWavePlate(np.radians(67.5))

# the linear polarizer 
polarizer = LinearPolarizer(0)

In [ ]:
detector = NoisyDetector(focal_grid, dark_current_rate=0, read_noise=0, flat_field=0, include_photon_noise=True)

In [ ]:
pupil_wavefront = Wavefront(aperture, wavelength)

pupil_wavefront.total_power = zero_magnitude_flux * 10**(-stellar_magnitude/2.5)
print("Total photon flux {:g}".format(pupil_wavefront.total_power))

Setting up the loop

In [ ]:

# total duration of the measurement
loop_duration = 8 # seconds

# number of measurements done (needs to be a multiple of 4)
Nevals = 4 # evaluations 

# integration time per modulation state 
delta_t = loop_duration / Nevals

# counter for the state of the modulation loop 
k = 0 

for t in np.linspace(0, loop_duration, Nevals):
    
    # selecting the modulator state
    if k == 0:
        HWP_modulator_state = HWP_state_1
    elif k == 1:
        HWP_modulator_state = HWP_state_2
    elif k == 2:
        HWP_modulator_state = HWP_state_3
    elif k == 3:
        HWP_modulator_state = HWP_state_4
        
    # we propagate the wavefront through the half-wave plate 
    pupil_wavefront_2 = HWP_modulator_state.forward(pupil_wavefront)
    
    # we propagate the wavefront through the linear polarizer 
    pupil_wavefront_3 = polarizer.forward(pupil_wavefront_3)
    
    # propagating the wavefront to the focal-plane
    focal_wavefront = propagator(pupil_wavefront_3)
    
    # integrating the detector
    detector.integrate(focal_wavefront, dt=delta_t)
    
    # reading the detector out 
    
    
        
        
    # Moving to the next modulator state 
    k += 1 

    # resetting the modulator 
    if k > 3:
        k = 0
    
    